The engine schematic (your puzzle input) consists of a visual representation of the engine. There are lots of numbers and symbols you don't really understand, but apparently any number adjacent to a symbol, even diagonally, is a "part number" and should be included in your sum. (Periods (.) do not count as a symbol.)


In [1]:
import numpy as np
import re

In [2]:
with open("input.txt") as file:
    lines = file.readlines()

In [3]:
list_of_lines = []
for line in lines:
    line_list = []
    for char in line.strip():
        line_list.append(char)
    list_of_lines.append(line_list)

In [4]:
array = np.array(list_of_lines)

In [5]:
# possible rules:
# if element is number and beside a symbol, then is True
# if element is number and beside a True, is True (to capture the other characters)

In [6]:
width, height = array.shape

In [7]:
def get_neighbour_indexes(i, j, width=140, height=140, only_side_neighbours=False):
    neighbours_indexes = []
    
    # if element is not at left side
    is_at_left = (j == 0)
    # if element is not at top
    is_at_top = (i == 0)
    # if element is not at bottom
    is_at_bottom = (i == (height - 1))
    # if element is not at right
    is_at_right = (j == (width - 1))

    
    if not is_at_left:
        neighbours_indexes.append((i, j-1)) # one to the left
    if not is_at_right:
        neighbours_indexes.append((i, j+1)) # one to the right
    
    if only_side_neighbours:
        return neighbours_indexes
    
    # these are wrong
    if not is_at_bottom and not is_at_left:
        neighbours_indexes.append((i+1, j-1)) # bottom left
    if not is_at_top and not is_at_left:
        neighbours_indexes.append((i-1, j-1)) # top left
    if not is_at_top:
        neighbours_indexes.append((i-1, j)) # above
    if not is_at_bottom:
        neighbours_indexes.append((i+1, j)) # below
    if not is_at_top and not is_at_right:
        neighbours_indexes.append((i-1, j+1)) # top right
    if not is_at_right and not is_at_bottom:
        neighbours_indexes.append((i+1, j+1)) # bottom right
    
    return neighbours_indexes

In [8]:
# checking get_neighbours_indexes is working as intended
tuple_list = [
    (0, 0),
    (139, 0),
    (100, 100),
    (0, 3),
]

for some_tuple in tuple_list:
    print(some_tuple)
    i, j = some_tuple
    print(get_neighbour_indexes(i, j))
    print(get_neighbour_indexes(i, j, only_side_neighbours=True))

(0, 0)
[(0, 1), (1, 0), (1, 1)]
[(0, 1)]
(139, 0)
[(139, 1), (138, 0), (138, 1)]
[(139, 1)]
(100, 100)
[(100, 99), (100, 101), (101, 99), (99, 99), (99, 100), (101, 100), (99, 101), (101, 101)]
[(100, 99), (100, 101)]
(0, 3)
[(0, 2), (0, 4), (1, 2), (1, 3), (1, 4)]
[(0, 2), (0, 4)]


In [9]:
def is_symbol(input_string):
    return input_string != '.' and not input_string.isalnum()

# checking is_symbol function
for input_string in ["a", ".", "6", "*", "+"]:
    print(input_string, is_symbol(input_string))

a False
. False
6 False
* True
+ True


In [10]:
def make_boolean_array(array, width=140, height=140):
    boolean_array = np.full((width, height), False)
    loop_count = 0
    while True:
        mean_value = boolean_array.mean()
        for i in range(width):
            # if i == 139:
            #     print(array[i])
            for j in range(height):
                if not array[i][j].isnumeric():
                    continue
                neighbour_indexes = get_neighbour_indexes(i, j, width=width, height=height)
                # check if any neighbour is a symbol
                # print((i, j), neighbour_indexes)
                any_neighbour_is_symbol = any(is_symbol(array[k, l]) for (k, l) in neighbour_indexes)
                if any_neighbour_is_symbol:
                    boolean_array[i, j] = True 
                
                # check is any neighbour is True (i.e. another number that neighbours a symbol)
                neighbour_indexes = get_neighbour_indexes(i, j, width, height, only_side_neighbours=True)
                # print((i, j), neighbour_indexes)
                any_neighbour_is_True = any(boolean_array[k, l] for (k, l) in neighbour_indexes)
                if any_neighbour_is_True:
                    boolean_array[i, j] = True
        loop_count+=1
        # print(mean_value, boolean_array.mean(), loop_count)
        # print(boolean_array.astype(int))
        if boolean_array.mean() == mean_value:
            break
    return boolean_array

In [12]:
def get_mask_of_array(array, boolean_array, width=140, height=140):
    numbers_array = np.full((140, 140),'.', dtype="<U10")
    for i in range(width):
        for j in range(height):
            if boolean_array[i][j] == True: # for some reason, is True isn't working
                numbers_array[i][j] = array[i][j].copy()
    
    return numbers_array

In [11]:
boolean_array = make_boolean_array(array=array)

In [13]:
numbers_array = get_mask_of_array(array, boolean_array)

In [14]:
list_of_numbers = [''.join(list(row)) for row in numbers_array]

# extracting ints from lines
list_of_number_values = [re.split(r'\D+', numbers) for numbers in list_of_numbers]

In [15]:
total_sum = 0
for number_values in list_of_number_values:
    list_sum = list(filter(lambda x: x.isnumeric(), number_values))
    list_sum = [int(value) for value in list_sum]
    list_sum = sum(list_sum)
    total_sum += list_sum

In [16]:
total_sum

514969

Part 2 - code could be definitely be made a bit more modular

In [17]:
def int_from_list_of_tuples(array, list_of_tuples):
    int_list = ''.join([array[i, j] for i, j in list_of_tuples])
    return int(int_list)

In [18]:
def find_gear_ratio_pairs(array):
    gear_ratio_sum = 0
    # find indexes of gears
    gears_indexes = np.argwhere(array == "*")
    print(f"{len(gears_indexes)} potential gears found")

    for gears_indexes_counter, (i, j) in enumerate(gears_indexes):
        # find neighbouring numbers
        neighbour_indexes = get_neighbour_indexes(i, j)
        number_indexes = [(k, l) for (k, l) in neighbour_indexes if array[k][l].isnumeric()]      
        
        # expand those number indexes to include all digits of the numbers
        neighbour_indexes = []
        for _ in range(6):
            for number_index in number_indexes:
                k, l = number_index
                neighbour_indexes_new = get_neighbour_indexes(k, l, only_side_neighbours=True)
                number_indexes_new = [(k, l) for k, l in neighbour_indexes_new if array[k][l].isnumeric()]
                number_indexes.extend(number_indexes_new)
                number_indexes = list(set(number_indexes))
        number_indexes.sort()
        
        # find groups of numbers
        first_number_indexes = []
        first_number_indexes.append(number_indexes[0])
        
        while True:
            # latest member of first set
            k, l = first_number_indexes[-1]
            if (k, l + 1) in number_indexes:
                first_number_indexes.append((k, l + 1))
            else:
                break
        
        if len(first_number_indexes) == len(number_indexes):
            # * not counted if only one number attached
            continue
        else:
            second_number_indexes = []
            second_number_indexes.append(number_indexes[len(first_number_indexes)])
            while True:
                k, l = second_number_indexes[-1]
                if (k, l + 1) in number_indexes:
                    second_number_indexes.append((k, l + 1))
                else:
                    break
        
        first_num = int_from_list_of_tuples(array, first_number_indexes)
        second_num = int_from_list_of_tuples(array, second_number_indexes)
        
        gear_ratio_sum += (first_num * second_num)
    return gear_ratio_sum
        
find_gear_ratio_pairs(array)

356 potential gears found


78915902